In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from pprint import pprint

prompt1 = ChatPromptTemplate.from_template("{country} 여행지(도시/국가)를 입력받아 대표 명소 1가지 추천해주세요.")

prompt2 = ChatPromptTemplate.from_template("{movie} 추천받은 명소의 상세 정보 제공해주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: "Drama" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"genre": "Drama"})
print("\n🔹 영화 줄거리 요약:")
pprint(response)

In [ ]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)